<a href="https://colab.research.google.com/github/Swarnlataaa/Generative-AI_demo/blob/main/BERT_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install streamlit transformers spacy
!python -m spacy download en_core_web_sm


In [ ]:
# app.py
import streamlit as st
import spacy
from transformers import BertForSequenceClassification, BertTokenizer
import torch

st.title("Automated Compliance Checker")

# Load Spacy model for text processing
nlp = spacy.load("en_core_web_sm")

# Load BERT model and tokenizer for compliance checking
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

user_input = st.text_area("Paste your legal document here:", height=400)
if st.button("Check Compliance"):
    # Perform compliance check and provide suggestions
    suggestions = check_compliance(user_input)
    st.write("Compliance Status:", suggestions)


In [ ]:
def check_compliance(text):
    # Tokenize the text using BERT tokenizer
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    labels = torch.tensor([1]).unsqueeze(0)  # Change the label according to your criterion

    # Use BERT for sequence classification
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
        loss, logits = outputs.loss, outputs.logits

    compliance_score = torch.softmax(logits, dim=1)[0][1].item()

    # Analyze the document using Spacy (example for named entity recognition)
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]

    # Provide suggestions based on the compliance score and entities found
    if compliance_score > 0.5:
        return "Compliant"
    else:
        suggestions = "Non-compliant. Check these entities: " + ", ".join(entities)
        return suggestions


In [ ]:
# !streamlit run app.py
